# MS Music: Mass Spectrometry Data Sonification

This notebook demonstrates the capabilities of the `ms_music` Python package, which allows users to convert mass spectrometry data from `.mzML` files into audio.

**Note:** This notebook assumes you have already installed the `ms_music` package and its dependencies (numpy, pandas, matchms, wavio, librosa, scipy, tqdm).

## 1. Setup and Imports

In [ ]:
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
from IPython.display import Audio, display
import os

# Assuming your package is named 'ms_music' and is importable
try:
    from ms_music import MSSonifier
    from ms_music import effects as ms_effects # Not strictly needed if calling via sonifier.apply_effect
except ImportError:
    print("MSSonifier class not found. Make sure the 'ms_music' package is installed and named correctly.")
    print("If you are running this from the source directory without installation, ensure paths are correct.")

## 2. Configuration

You'll need an `.mzML` file to run this demonstration. 

**Action Required:**
1. Create a folder named `sample_data` in the same directory as this notebook.
2. Place your `.mzML` file into the `sample_data` folder.
3. Update the `mzml_filename` variable below with the name of your file.

In [ ]:
mzml_filename = ".mzML"  # Replace with your actual mzML file name
sample_data_dir = "sample_data_dir"
output_audio_dir = "output_audio_dir"

# Create directories if they don't exist
os.makedirs(sample_data_dir, exist_ok=True)
os.makedirs(output_audio_dir, exist_ok=True)

mzml_filepath = os.path.join(sample_data_dir, mzml_filename)

# Check if the file exists
if not os.path.exists(mzml_filepath):
    print(f"ERROR: mzML file not found at {mzml_filepath}")
    print("Please place your mzML file in the 'sample_data' directory and update 'mzml_filename'.")
else:
    print(f"Using mzML file: {mzml_filepath}")

# Sonification parameters
MS_LEVEL = 1                # MS level to process (1 or 2)
TOTAL_DURATION_MIN = 2   # Desired output audio duration in minutes (keep it very short for demos)
SAMPLE_RATE = 44100         # Audio sample rate

## 3. Initialize the Sonifier and Load Data

In [ ]:
sonifier = None # Initialize sonifier to None
if os.path.exists(mzml_filepath):
    sonifier = MSSonifier(
        filepath=mzml_filepath,
        ms_level=MS_LEVEL,
        total_duration_minutes=TOTAL_DURATION_MIN,
        sample_rate=SAMPLE_RATE
    )
    sonifier.load_and_preprocess_data()
else:
    print("Skipping sonifier initialization as mzML file is not found.")

## 4. Generate Base Audio for Effects Demonstration

We'll generate one or two base audio files using different sonification methods. These will then be used as input for showcasing the effects. We'll primarily use the ADSR output for effects demonstration as it often provides a clearer canvas for hearing effect changes.

In [ ]:
base_gradient_audio = None

if sonifier and sonifier.processed_spectra_dfs is not None:
    print("\n--- Generating Base Gradient Audio ---")
    sonifier.sonify(method='gradient', method_params={'overlap_percentage': 0.05})
    base_gradient_audio = sonifier.get_current_audio()
    if base_gradient_audio is not None and base_gradient_audio.size > 0:
        # Normalize base_gradient_audio so that it's amplitude is between -1 and 1
        if base_gradient_audio is not None:
            max_amplitude = np.max(np.abs(base_gradient_audio))
            if max_amplitude > 0:
                base_gradient_audio /= max_amplitude
                print("Base gradient audio normalized.")
        output_base_gradient_path = os.path.join(output_audio_dir, "base_audio_gradient.wav")
        sonifier.save_audio(output_base_gradient_path)
        print(f"Base Gradient audio saved to {output_base_gradient_path}")
        display(Audio(data=base_gradient_audio, rate=SAMPLE_RATE, normalize=True))
    else:
        print("Base Gradient sonification did not produce audio.")
else:
    print("Sonifier not ready. Cannot generate base audio.")

In [ ]:
base_adsr_audio = None

if sonifier and sonifier.processed_spectra_dfs is not None:
    print("\n--- Generating Base ADSR Audio ---")
    adsr_settings = {
        'attack_time_pc': 0.02, 'decay_time_pc': 0.05,
        'sustain_level_pc': 0.7, 'release_time_pc': 0.1,
        'randomize': False
    }
    sonifier.sonify(method='adsr', method_params={'adsr_settings': adsr_settings})
    base_adsr_audio = sonifier.get_current_audio()
    if base_adsr_audio is not None and base_adsr_audio.size > 0:
        output_base_adsr_path = os.path.join(output_audio_dir, "base_audio_adsr.wav")
        sonifier.save_audio(output_base_adsr_path)
        print(f"Base ADSR audio saved to {output_base_adsr_path}")
        display(Audio(data=base_adsr_audio, rate=SAMPLE_RATE, normalize=True))
    else:
        print("Base ADSR sonification did not produce audio.")
else:
    print("Sonifier not ready. Cannot generate base audio.")

## 5. Applying Audio Effects

Now, we'll demonstrate various audio effects. For each effect, we will reload the `base_adsr_audio` into the sonifier's current buffer to ensure we're applying the effect to a consistent input, making it easier to discern the impact of each individual effect.

### 5.1. Standard Audio Effects

In [ ]:
# Helper function to reset sonifier's audio and apply/save/display an effect
def demonstrate_effect(effect_name, effect_params, base_audio, filename_suffix):
    if sonifier is None or base_audio is None or base_audio.size == 0:
        print(f"Skipping effect '{effect_name}': Sonifier or base audio not ready.")
        return

    print(f"\n--- Demonstrating Effect: {effect_name} ---")
    sonifier.current_audio_data = np.copy(base_audio) # Reset to base audio
    
    sonifier.apply_effect(effect_name, effect_params=effect_params)
    effect_output_audio = sonifier.get_current_audio()
    
    if effect_output_audio is not None and effect_output_audio.size > 0:
        output_path = os.path.join(output_audio_dir, f"effect_{filename_suffix}.wav")
        sonifier.save_audio(output_path)
        print(f"{effect_name} audio saved to {output_path}")
        display(Audio(data=effect_output_audio, rate=SAMPLE_RATE, normalize=False))
    else:
        print(f"{effect_name} did not produce audio or an error occurred.")

# Ensure base_gradient_audio is available for the following demonstrations
if 'base_gradient_audio' not in locals() or base_gradient_audio is None:
    print("Base ADSR audio is not available. Effects demonstration will be skipped.")
else:
    # HPSS (Harmonic)
    demonstrate_effect('hpss', 
                       {'margin': 16, 'harmonic': True, 'percussive': False}, 
                       base_gradient_audio, 
                       'hpss_harmonic')

    # HPSS (Percussive)
    demonstrate_effect('hpss', 
                       {'margin': 16, 'harmonic': False, 'percussive': True}, 
                       base_gradient_audio, 
                       'hpss_percussive')

    # Notch Filter
    demonstrate_effect('notch_filter', 
                       {'notch_freq': 1000, 'quality_factor': 10}, 
                       base_gradient_audio, 
                       'notch_filter_1kHz')

    # Butterworth Low-pass Filter
    demonstrate_effect('butterworth_filter', 
                       {'cutoff_freq': 800, 'btype': 'low', 'order': 4}, 
                       base_gradient_audio, 
                       'butterworth_lowpass_800Hz')

    # Butterworth Band-pass Filter
    demonstrate_effect('butterworth_filter', 
                       {'cutoff_freq': (300, 1200), 'btype': 'bandpass', 'order': 3, 'gustafson_method': True}, 
                       base_gradient_audio, 
                       'butterworth_bandpass_300_1200Hz')

    # Chebyshev Type I Low-pass Filter
    demonstrate_effect('chebyshev1_filter', 
                       {'cutoff_freq': 1500, 'ripple_db': 1, 'btype': 'low', 'order': 4}, 
                       base_gradient_audio, 
                       'chebyshev1_lowpass_1500Hz_1dB')

### 5.2. Experimental Audio Effects

These effects are derived from the user's original `complex.py` script and are marked as experimental. They might be unstable, produce unexpected results, or be computationally intensive. Use with caution and for exploratory purposes. Some might effectively be no-ops if their internal logic was flagged as non-functional.

In [ ]:
if 'base_gradient_audio' not in locals() or base_gradient_audio is None:
    print("Base ADSR audio is not available. Experimental effects demonstration will be skipped.")
else:
    # Experimental PLL Filter (likely a no-op returning original audio)
    demonstrate_effect('experimental_pll_filter', 
                       {'loop_gain': 0.01, 'loop_bandwidth': 50.0, 'center_freq': 500.0, 'freq_deviation': 20.0}, 
                       base_gradient_audio, 
                       'exp_pll_filter')

    # Experimental LMS Modulation
    demonstrate_effect('experimental_lms_modulation', 
                       {'n_segments': 50, 'mu': 0.02}, # Reduced n_segments for faster demo
                       base_gradient_audio, 
                       'exp_lms_modulation')

    # Experimental Spectral Analysis FM
    demonstrate_effect('experimental_spectral_analysis_fm', 
                       {}, # No specific params other than audio_data and sample_rate
                       base_gradient_audio, 
                       'exp_spectral_analysis_fm')

    # Experimental Phase Vocoder Modulation (placeholder implementation)
    demonstrate_effect('experimental_phase_vocoder_modulation', 
                       {'n_fft': 2048, 'hop_length': 512}, 
                       base_gradient_audio, 
                       'exp_phase_vocoder_mod')

    # Experimental FFT Filter
    demonstrate_effect('experimental_fft_filter', 
                       {'threshold_abs_coeff': 2.0}, 
                       base_gradient_audio, 
                       'exp_fft_filter_thresh2')

## 6. Visualizing Spectrograms

You can use `librosa.display` to visualize the spectrogram of the generated audio. We'll plot the base ADSR audio and one of the processed outputs.

In [ ]:
def plot_spectrogram(audio_data, sr, title="Spectrogram"):
    if audio_data is None or audio_data.size == 0:
        print(f"Cannot plot spectrogram for {title}: No audio data.")
        return
    # Ensure audio is float for STFT
    audio_float = audio_data.astype(np.float32)
    # Normalize if it's int16 from saving/loading without explicit normalization for display
    if np.issubdtype(audio_float.dtype, np.integer):
        audio_float = audio_float / (2**15 -1) # Basic normalization for int16
        
    X = librosa.stft(audio_float)
    Xdb = librosa.amplitude_to_db(abs(X))
    plt.figure(figsize=(14, 5))
    librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='log') # Using log y-axis
    plt.title(title)
    plt.colorbar(format='%+2.0f dB')
    plt.show()

if 'base_gradient_audio' in locals() and base_gradient_audio is not None:
    plot_spectrogram(base_gradient_audio, SAMPLE_RATE, "Spectrogram of Base gradient Audio")

# Example: Plot spectrogram of the Butterworth low-pass filtered audio
if sonifier and os.path.exists(os.path.join(output_audio_dir, 'effect_butterworth_lowpass_800Hz.wav')):
    # Load the saved audio to ensure we plot what was displayed
    try:
        butterworth_audio_loaded, sr_loaded = librosa.load(os.path.join(output_audio_dir, 'effect_butterworth_lowpass_800Hz.wav'), sr=SAMPLE_RATE)
        plot_spectrogram(butterworth_audio_loaded, SAMPLE_RATE, "Spectrogram of Butterworth Low-Pass Filtered ADSR Audio")
    except Exception as e:
        print(f"Could not load or plot spectrogram for Butterworth filtered audio: {e}")
